In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

LSTM_UNITS = 1

#USE RANDOM IMAGES TO SET UP WORKING EXAMPLE
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.conv = nn.Conv2d(3,1,3) #input 50x50x3
        self.pool = nn.MaxPool2d(2,2) #input 48x48 output 24x24
        
        self.lstm = nn.LSTM(24*24,24*24,LSTM_UNITS)
        self.fc1 = nn.Linear(24*24,120)
        self.fc2 = nn.Linear(120,3)
        
        #initialize hidden states of LSTM
        self._hidden = (torch.randn(1, 1, 24*24), torch.randn(1, 1, 24*24))
        #print("Hidden:", _hidden)
        
    def forward(self,x):
        #F. from torch.nn.functional
        print("Input:", x.size())
        x = x.float() #necessary for some reason
        x = self.pool(F.relu(self.conv(x)))
        x = x.view([24*24])
        print("LSTM:", len(x))
        x, _hidden = self.lstm(x.view(1, 1, -1), self._hidden)
        #x = self.lstm(x.view(1,1,-1),_hidden) 
        # RELU not possible?
        # https://discuss.pytorch.org/t/one-to-one-lstm-input-format-unclear-undocumented-threshold-argument-error/11878
        x = x.view([1,24*24])
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
print("Class defined")

test_images = torch.tensor(np.random.rand(1,3,50,50))
#print(image)
time_steps = 100
batch_size = 1
in_size = np.size((24,24)) #5x5 input image
#print(in_size)
#autotest_input_seq = torch.autograd.Variable(torch.randn(time_steps, batch_size, in_size))
#print(np.size(test_images[0]))

#training_loader = torch.utils.data.DataLoader(
#    torch.from_numpy(test_images),
#    batch_size=1, 
#    shuffle=False, 
#    num_workers = 0)
#print('Training loader created')

#TRAINING
test_net = TEST_CNN_LSTM()
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = optim.SGD(test_net.parameters(), lr=0.01, momentum=0.9)


print('Start training...')
for epoch in range(2): 
    print("Epoch:",epoch)
    #TRAINING DATA
    running_loss = 0.0

    print("|----------|")
    for i in range(1):
        
        #inputs, labels = data
        inputs = test_images
        labels = torch.tensor([2])
        ### WRONG LABELING FOR CROSS ENTROPY LOSS
        
        optimizer.zero_grad() 
        outputs = test_net(inputs)
        print("Out:", len(outputs), outputs)
        print("Labels:", len(labels), labels)
        loss = criterion(outputs, labels)
        loss.backward() 
        
        optimizer.step()


        # print statistics
        running_loss += loss.item()
        
print('...Training finished')




Class defined
Start training...
('Epoch:', 0)
|----------|
('Input:', torch.Size([1, 3, 50, 50]))
('LSTM:', 576)
('Out:', 1, tensor([[ 0.1059, -0.0036, -0.0502]], grad_fn=<AddmmBackward>))
('Labels:', 1, tensor([2]))
('Epoch:', 1)
|----------|
('Input:', torch.Size([1, 3, 50, 50]))
('LSTM:', 576)
('Out:', 1, tensor([[ 0.0650, -0.0253,  0.0227]], grad_fn=<AddmmBackward>))
('Labels:', 1, tensor([2]))
...Training finished


In [56]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(input)
print(len(input),len(target))
print(target)

tensor([[ 0.1740,  0.7721,  0.9229, -1.2416, -0.2742],
        [ 1.0118, -1.1599, -0.6177,  1.0337, -1.4971],
        [-0.3328,  1.4890, -0.7336, -0.5394, -1.1171]], requires_grad=True)
(3, 3)
tensor([4, 4, 0])
